In [318]:
import importlib
from functools import partial

from torch import nn
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import copy


from decision_learning.data.shortest_path_grid import genData
from decision_learning.modeling.loss import SPOPlus
from decision_learning.modeling.models import LinearRegression
from decision_learning.modeling.val_metrics import decision_regret
from decision_learning.modeling.train import train, calc_test_regret, init_loss_data_pretraining, filter_kwargs

In [23]:
# Only for on the fly reloading of changing modules
# import decision_learning.data.shortest_path_grid
# importlib.reload(decision_learning.data.shortest_path_grid)
# import decision_learning.modeling.val_metrics
# importlib.reload(decision_learning.modeling.val_metrics)

# import decision_learning.modeling.models
# importlib.reload(decision_learning.modeling.models)
# from decision_learning.modeling.models import LinearRegression
# import decision_learning.modeling.loss
# importlib.reload(decision_learning.modeling.loss)
# from decision_learning.modeling.loss import SPOPlus
# import decision_learning.modeling.train
# importlib.reload(decision_learning.modeling.train)
# from decision_learning.modeling.train import train, calc_test_regret

<module 'decision_learning.modeling.train' from '/home1/yongpeng/decision-focused-learning/src/decision_learning/modeling/train.py'>

# Optimization Solver

In [105]:
def shortest_path_solver(costs, size, sens = 1e-4):
    # Forward Pass
    starting_ind = 0
    starting_ind_c = 0
    samples = costs.shape[0]
    V_arr = torch.zeros(samples, size ** 2)
    for i in range(0, 2 * (size - 1)):
        num_nodes = min(i + 1, 9 - i)
        num_nodes_next = min(i + 2, 9 - i - 1)
        num_arcs = 2 * (max(num_nodes, num_nodes_next) - 1)
        V_1 = V_arr[:, starting_ind:starting_ind + num_nodes]
        layer_costs = costs[:, starting_ind_c:starting_ind_c + num_arcs]
        l_costs = layer_costs[:, 0::2]
        r_costs = layer_costs[:, 1::2]
        next_V_val_l = torch.ones(samples, num_nodes_next) * float('inf')
        next_V_val_r = torch.ones(samples, num_nodes_next) * float('inf')
        if num_nodes_next > num_nodes:
            next_V_val_l[:, :num_nodes_next - 1] = V_1 + l_costs
            next_V_val_r[:, 1:num_nodes_next] = V_1 + r_costs
        else:
            next_V_val_l = V_1[:, :num_nodes_next] + l_costs
            next_V_val_r = V_1[:, 1:num_nodes_next + 1] + r_costs
        next_V_val = torch.minimum(next_V_val_l, next_V_val_r)
        V_arr[:, starting_ind + num_nodes:starting_ind + num_nodes + num_nodes_next] = next_V_val

        starting_ind += num_nodes
        starting_ind_c += num_arcs

    # Backward Pass
    starting_ind = size ** 2
    starting_ind_c = costs.shape[1]
    prev_act = torch.ones(samples, 1)
    sol = torch.zeros(costs.shape)
    for i in range(2 * (size - 1), 0, -1):
        num_nodes = min(i + 1, 9 - i)
        num_nodes_next = min(i, 9 - i + 1)
        V_1 = V_arr[:, starting_ind - num_nodes:starting_ind]
        V_2 = V_arr[:, starting_ind - num_nodes - num_nodes_next:starting_ind - num_nodes]

        num_arcs = 2 * (max(num_nodes, num_nodes_next) - 1)
        layer_costs = costs[:, starting_ind_c - num_arcs: starting_ind_c]

        if num_nodes < num_nodes_next:
            l_cs_res = ((V_2[:, :num_nodes_next - 1] - V_1 + layer_costs[:, ::2]) < sens) * prev_act
            r_cs_res = ((V_2[:, 1:num_nodes_next] - V_1 + layer_costs[:, 1::2]) < sens) * prev_act
            prev_act = torch.zeros(V_2.shape)
            prev_act[:, :num_nodes_next - 1] += l_cs_res
            prev_act[:, 1:num_nodes_next] += r_cs_res
        else:
            l_cs_res = ((V_2 - V_1[:, :num_nodes - 1] + layer_costs[:, ::2]) < sens) * prev_act[:, :num_nodes - 1]
            r_cs_res = ((V_2 - V_1[:, 1:num_nodes] + layer_costs[:, 1::2]) < sens) * prev_act[:, 1:num_nodes]
            prev_act = torch.zeros(V_2.shape)
            prev_act += l_cs_res
            prev_act += r_cs_res
        cs = torch.zeros(layer_costs.shape)
        cs[:, ::2] = l_cs_res
        cs[:, 1::2] = r_cs_res
        sol[:, starting_ind_c - num_arcs: starting_ind_c] = cs

        starting_ind = starting_ind - num_nodes
        starting_ind_c = starting_ind_c - num_arcs
    # Dimension (samples, num edges)
    obj = torch.sum(sol * costs, axis=1)
    # Dimension (samples, 1)
    return sol.to(torch.float32), obj.reshape(-1,1).to(torch.float32)

# Verify Previous Results

# Compare Input Data

### Shared Inputs to both old and new data

In [41]:
torch.manual_seed(105)
indices_arr = torch.randperm(100000)
indices_arr_test = torch.randperm(100000)

sim = 0
n_arr = [200, 400, 800, 1600]
ep_arr = ['unif', 'normal']
trials = 100

exp_arr = []
for n in n_arr:
    for ep in ep_arr:
        for t in range(trials):
            exp_arr.append([n, ep, t])

# setup
exp = exp_arr[0]
ep_type = exp[1]
trial = exp[2]

# generate data
grid = (5, 5)  # grid size
num_data = exp[0]  # number of training data
num_feat = 5  # size of feature
deg = 6  # polynomial degree
e = .3  # noise width

print(num_data, ep_type, trial)

200 unif 0


### New Input Data

In [87]:
import decision_learning.data.shortest_path_grid
importlib.reload(decision_learning.data.shortest_path_grid)
from decision_learning.data.shortest_path_grid import genData

In [88]:
# path planting for shortest path example
planted_good_pwl_params = {'slope0':0, 
                    'int0':2,
                    'slope1':0, 
                    'int1':2}
planted_bad_pwl_params = {'slope0':4, 
                    'int0':0,
                    'slope1':0, 
                    'int1':2.2}

plant_edge = True

In [432]:
generated_data = genData(num_data=num_data+200,
        num_features=num_feat, 
        grid=grid, 
        deg=deg, 
        noise_type=ep_type,
        noise_width=e,
        seed=indices_arr[trial],     
        plant_edges=plant_edge,
        planted_good_pwl_params=planted_good_pwl_params,
        planted_bad_pwl_params=planted_bad_pwl_params)
sol, obj = shortest_path_solver(costs=generated_data['cost'], size=5)
final_data = {'X':generated_data['feat'],
              'true_cost':generated_data['cost'],
              'true_sol':sol,
              'true_obj':obj}

# Splitting each input in the same way
train_dict = {}
val_dict = {}

for key, value in final_data.items():
    train_data, val_data = train_test_split(value, test_size=200, random_state=42)
    train_dict[key] = train_data
    val_dict[key] = val_data
    
# test data
generated_data_test = genData(num_data=10000,
        num_features=num_feat, 
        grid=grid, 
        deg=deg, 
        noise_type=ep_type,
        noise_width=e,
        seed=indices_arr_test[trial],     
        plant_edges=plant_edge,
        planted_good_pwl_params=planted_good_pwl_params,
        planted_bad_pwl_params=planted_bad_pwl_params)
sol_test, obj_test = shortest_path_solver(costs=generated_data_test['cost_true'], size=5)
final_data_test = {'X':generated_data_test['feat'],
              'true_cost':generated_data_test['cost_true'],
              'true_sol':sol_test,
              'true_obj':obj_test}

2024-11-13 16:23:53,068 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 10 11 12 13 14 17 18 19 20 21 22 25 26 27 28 29 32 33 34 37]
2024-11-13 16:23:53,068 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 10 11 12 13 14 17 18 19 20 21 22 25 26 27 28 29 32 33 34 37]
2024-11-13 16:23:53,069 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.0
2024-11-13 16:23:53,069 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.0
2024-11-13 16:23:53,069 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.55
2024-11-13 16:23:53,069 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.55
2024-11-13 16:23:53,096 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 

In [431]:
generated_data_test.keys()

dict_keys(['feat', 'cost_noplant', 'cost_true', 'cost', 'epsilon'])

### Old Input Data

In [234]:
import sys
sys.path.append('/home1/yongpeng/decision-focused-learning/experiments/shortest_path_grid_exp/')
import sp_grid_mike_neurips
importlib.reload(sp_grid_mike_neurips)
import sp_grid_mike_neurips

In [131]:
feat, cost_true, cost, eps = sp_grid_mike_neurips.genData(num_data + 200, 
                                                     num_feat, 
                                                     grid, 
                                                     ep_type, 
                                                     deg, 
                                                     e, 
                                                     seed=indices_arr[trial])
x_train, x_val, c_train, c_val = train_test_split(feat, cost, test_size=200, random_state=42)
x_test, c_test, c_hat_test, eps_test = sp_grid_mike_neurips.genData(10000, 
                                                          num_feat, 
                                                          grid, 
                                                          ep_type, 
                                                          deg, 
                                                          e, 
                                                          seed=indices_arr_test[trial])

# train
sol_tr, obj_tr = shortest_path_solver(costs=c_train, size=5)
train_dict_old = {'X':x_train[:,:-1],
              'true_cost':c_train,
              'true_sol':sol_tr,
              'true_obj':obj_tr}
# val
sol_v, obj_v = shortest_path_solver(costs=c_val, size=5)
val_dict_old = {'X':x_val[:,:-1],
              'true_cost':c_val,
              'true_sol':sol_v,
              'true_obj':obj_v}

# test
sol_te, obj_te = shortest_path_solver(costs=c_hat_test, size=5)
final_data_te = {'X':x_test[:,:-1],
              'true_cost':c_hat_test,
              'true_sol':sol_te,
              'true_obj':obj_te}

### Compare the old and new data inputs

#### Costs before noise

In [93]:
(torch.tensor(generated_data['cost_true'], dtype=torch.float32) == cost_true).all()

tensor(True)

#### Costs After Noise

In [94]:
(torch.tensor(generated_data['cost'], dtype=torch.float32) == cost).all()

tensor(True)

#### Training Data

In [113]:
for k in train_dict.keys():
    print(k)
    print(torch.allclose(torch.tensor(train_dict[k], dtype=torch.float32),
                   train_dict_old[k]))
                   

X
True
true_cost
True
true_sol
True
true_obj
True


/tmp/SLURM_27423203/ipykernel_37376/1802200100.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(torch.allclose(torch.tensor(train_dict[k], dtype=torch.float32),


#### Validation Data

In [116]:
for k in val_dict.keys():
    print(k)
    print(torch.allclose(torch.tensor(val_dict[k], dtype=torch.float32),
                   val_dict_old[k]))

X
True
true_cost
True
true_sol
True
true_obj
True


/tmp/SLURM_27423203/ipykernel_37376/2673537216.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(torch.allclose(torch.tensor(val_dict[k], dtype=torch.float32),


#### Test Data

In [132]:
for k in final_data_test.keys():
    print(k)
    print(torch.allclose(torch.tensor(final_data_test[k], dtype=torch.float32),
                   final_data_te[k]))

X
True
true_cost
True
true_sol
True
true_obj
True


/tmp/SLURM_27423203/ipykernel_37376/4035419391.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(torch.allclose(torch.tensor(final_data_test[k], dtype=torch.float32),


# Data All the Same!

# Loss Function Difference?

In [291]:
predmodel = LinearRegression(input_dim=feat.shape[1],
                 output_dim=train_dict['true_cost'].shape[1])
optimizer = torch.optim.Adam(pred_model.parameters(), lr=0.01)

#### Loss Calc

New Loss Function

In [292]:
new_loss_fn = SPOPlus(optmodel=optmodel)

In [293]:
cp = predmodel(train_dict_old['X'])
new_loss = new_loss_fn(cp,
            true_cost=train_dict_old['true_cost'],
            true_sol=train_dict_old['true_sol'], 
            true_obj=train_dict_old['true_obj'])
new_loss

tensor(10.9612, grad_fn=<MeanBackward0>)

Gradient

In [294]:
new_loss.backward()

In [295]:
# Check gradients

print("Gradients of the model's parameters:")
for name, param in predmodel.named_parameters():
    print(f"{name} gradient:\n{param.grad[0]}")

Gradients of the model's parameters:
linear.weight gradient:
tensor([-0.1378,  0.3801, -0.2576,  0.1089,  0.1560, -1.0474, -0.5900])
linear.bias gradient:
-0.5899999737739563


In [297]:
# Reset gradients
for param in predmodel.parameters():
    if param.grad is not None:
        param.grad.zero_()

Old Loss

In [334]:
import sp_grid_mike_neurips
importlib.reload(sp_grid_mike_neurips)
from sp_grid_mike_neurips import SPOPlus2, regret_func, trainModel, regret_func

In [299]:
optmodel_old = sp_grid_mike_neurips.shortestPathModel()
old_loss_fn = SPOPlus2(optmodel_old)

cp = predmodel(train_dict_old['X'])
old_loss = old_loss_fn(cp,
            true_cost=train_dict_old['true_cost'],
            true_sol=train_dict_old['true_sol'], 
            true_obj=train_dict_old['true_obj'])
old_loss

Num of cores: 1


tensor(10.9612, grad_fn=<MeanBackward0>)

In [300]:
old_loss.backward()

In [301]:
# Check gradients
print("Gradients of the model's parameters:")
for name, param in predmodel.named_parameters():
    print(f"{name} gradient:\n{param.grad[0]}")

Gradients of the model's parameters:
linear.weight gradient:
tensor([-0.1378,  0.3801, -0.2576,  0.1089,  0.1560, -1.0474, -0.5900])
linear.bias gradient:
-0.5899999737739563


In [286]:
# Reset gradients
for param in predmodel.parameters():
    if param.grad is not None:
        param.grad.zero_()

Gradients also the same

In [313]:
predmodel = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])

Baby New Training Loop

In [433]:
new_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])
new_model.linear = copy.deepcopy(predmodel.linear)

optmodel = partial(shortest_path_solver,size=5)
loss_fn = SPOPlus(optmodel=optmodel)

preset_params = {'optmodel': optmodel, 'minimize': True}
val_metric = partial(decision_regret, **preset_params)

optimizer = torch.optim.Adam(new_model.parameters(), lr=0.01)
train_dataset, train_loader = init_loss_data_pretraining(train_dict, {'batch_size':200, 'shuffle':True})  # training data        
val_dataset, val_loader = init_loss_data_pretraining(val_dict, {'batch_size':200, 'shuffle':True})  # validation data
    

In [434]:
metrics = []
for epoch in range(100):
    epoch_losses = []
    new_model.train() # beginning of epoch, set model to training mode
    for batch_idx, batch in enumerate(tqdm(train_loader)):    
                
        pred = new_model(batch['X'])
        
        batch = filter_kwargs(loss_fn.forward, batch) # filter out only valid arguments for the loss function                         
        loss = loss_fn(pred, **batch) # we assume the first argument of the loss function is the prediction, while the rest can be passed as kwargs
        
        # standard backwards pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())        
    
    print(new_model.linear.weight[0])
    new_model.eval()
    all_preds = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(val_loader)):                    
            pred = new_model(batch['X'])                    
            all_preds.append(pred)
    all_preds = torch.cat(all_preds, dim=0)
    
    val_data_dict = filter_kwargs(val_metric, val_dict) # filter out only valid arguments for the val metric    
    val_loss = val_metric(all_preds, **val_data_dict)

    cur_metric = {'epoch': epoch, 
                    'train_loss': np.mean(epoch_losses), 
                    'val_metric': val_loss}
    metrics.append(cur_metric)
    
    print(f'epoch: {epoch}, train_loss: {np.mean(epoch_losses)}, val_metric: {val_loss}')
metrics = pd.DataFrame(metrics)

100%|██████████| 1/1 [00:00<00:00, 250.26it/s]


tensor([-0.3795,  0.1344, -0.3751, -0.3201, -0.0209,  0.3107],
       grad_fn=<SelectBackward0>)


100%|██████████| 1/1 [00:00<00:00, 665.02it/s]


epoch: 0, train_loss: 17.493484497070312, val_metric: 0.5201238330848681


100%|██████████| 1/1 [00:00<00:00, 328.01it/s]


...

epoch: 98, train_loss: 4.190544605255127, val_metric: 0.2141065085934349


100%|██████████| 1/1 [00:00<00:00, 330.57it/s]


tensor([-0.3828, -0.0885, -0.1566, -0.3408, -0.1249,  0.9281],
       grad_fn=<SelectBackward0>)


100%|██████████| 1/1 [00:00<00:00, 705.52it/s]

epoch: 99, train_loss: 4.183767318725586, val_metric: 0.20329048614971318


In [356]:
epoch_losses

[17.493484497070312]

Baby Old Training Loop

In [383]:
import pyepo

feat, cost_true, cost, eps = sp_grid_mike_neurips.genData(num_data + 200, 
                                                     num_feat, 
                                                     grid, 
                                                     ep_type, 
                                                     deg, 
                                                     e, 
                                                     seed=indices_arr[trial])
x_train, x_val, c_train, c_val = train_test_split(feat, cost, test_size=200, random_state=42)
x_test, c_test, c_hat_test, eps_test = sp_grid_mike_neurips.genData(10000, 
                                                          num_feat, 
                                                          grid, 
                                                          ep_type, 
                                                          deg, 
                                                          e, 
                                                          seed=indices_arr_test[trial])

old_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])
old_model.linear = copy.deepcopy(predmodel.linear)
print(old_model.linear.weight[0])

optmodel_old = sp_grid_mike_neurips.shortestPathModel()
old_loss_fn = SPOPlus2(optmodel_old)

optimizer = torch.optim.Adam(old_model.parameters(), lr=0.01)
# build dataset
dataset = pyepo.data.dataset.optDataset(optmodel_old, x_train[:,:-1], c_train)
loader_train = DataLoader(dataset, batch_size=200, shuffle=True)    

tensor([-0.3895,  0.1444, -0.3851, -0.3101, -0.0109,  0.3007],
       grad_fn=<SelectBackward0>)
Num of cores: 1
Optimizing for optDataset...


100%|██████████| 200/200 [00:00<00:00, 1052.83it/s]


In [384]:
# train mode
old_model.train()
# init log
loss_log = [['trial', 'n', 'epoch', 'loss_name', 'regret', 'val_regret']]
loss_name = 'SPO+'
for epoch in range(100):
    
    batch_loss = []
    for i, data in enumerate(loader_train):
        x, c, w, z = data
        w = w.reshape(w.shape[0], -1)
        z = z.reshape(z.shape[0], -1)
        
        cp = old_model(x)
        loss = old_loss_fn(cp, c, w, z)
        

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())

    print(old_model.linear.weight[0])
    train_regret = sum(batch_loss)
    regret = regret_func(old_model, torch.FloatTensor(x_test[:,:-1]), torch.FloatTensor(c_test))
    val_regret = regret_func(old_model, torch.FloatTensor(x_val[:,:-1]), torch.FloatTensor(c_val))

    loss_log.append([trial, num_data, epoch, loss_name, regret, val_regret])

    print(
        "Epoch {:2},  Train_Regret: {}, Val_Regret: {:7.4f}%, Regret: {:7.4f}%".format(epoch + 1, train_regret,
                                                                                           val_regret * 100,
                                                                                           regret * 100))


tensor([-0.3795,  0.1344, -0.3751, -0.3201, -0.0209,  0.3107],
       grad_fn=<SelectBackward0>)
Epoch  1,  Train_Regret: 17.493484497070312, Val_Regret: 53.8790%, Regret: 49.1503%
tensor([-0.3714,  0.1244, -0.3651, -0.3298, -0.0309,  0.3202],
       grad_fn=<SelectBackward0>)
Epoch  2,  Train_Regret: 16.78026580810547, Val_Regret: 51.8425%, Regret: 47.1263%
tensor([-0.3660,  0.1144, -0.3551, -0.3358, -0.0407,  0.3299],
       grad_fn=<SelectBackward0>)
Epoch  3,  Train_Regret: 16.084392547607422, Val_Regret: 49.8255%, Regret: 44.9284%
tensor([-0.3653,  0.1044, -0.3451, -0.3390, -0.0503,  0.3397],
       grad_fn=<SelectBackward0>)
Epoch  4,  Train_Regret: 15.414484977722168, Val_Regret: 45.8209%, Regret: 42.1496%
tensor([-0.3689,  0.0945, -0.3351, -0.3377, -0.0599,  0.3494],
       grad_fn=<SelectBackward0>)
Epoch  5,  Train_Regret: 14.766684532165527, Val_Regret: 43.3121%, Regret: 39.2565%
tensor([-0.3657,  0.0845, -0.3251, -0.3331, -0.0693,  0.3591],
       grad_fn=<SelectBackward0>)

In [385]:
regret

0.04638737440244818

### Verify the Regret Calc

Old Regret

In [435]:
old_reg_func_old_mod = regret_func(old_model, torch.FloatTensor(x_test[:,:-1]), torch.FloatTensor(c_hat_test))
old_reg_func_new_mod = regret_func(new_model, torch.FloatTensor(x_test[:,:-1]), torch.FloatTensor(c_hat_test))

old_reg_func_old_mod, old_reg_func_new_mod

(0.08560629393454147, 0.08560629393454147)

In [427]:
c_hat_test, final_data_test['true_cost']

(tensor([[1.3985, 1.7580, 3.2470,  ..., 3.2330, 1.5450, 1.7464],
         [2.3881, 1.4507, 2.2957,  ..., 2.9421, 2.2133, 1.5494],
         [1.8591, 1.8944, 4.6037,  ..., 2.9546, 2.8156, 2.2623],
         ...,
         [2.0214, 2.2908, 1.9479,  ..., 2.8776, 1.7721, 1.4501],
         [1.7199, 1.8696, 3.1003,  ..., 3.4000, 1.7542, 2.1117],
         [2.5083, 2.4716, 2.1675,  ..., 4.1673, 1.6614, 1.6436]]),
 array([[1.39848442, 1.75802185, 3.24695078, ..., 3.23298268, 1.54500231,
         1.7464299 ],
        [2.38808872, 1.45070565, 2.29568997, ..., 2.94207769, 2.21333371,
         1.54944921],
        [1.85908374, 1.89436551, 4.60370222, ..., 2.95460682, 2.81563494,
         2.26229281],
        ...,
        [2.02139322, 2.2907848 , 1.94794872, ..., 2.87761556, 1.77207911,
         1.4501351 ],
        [1.7198845 , 1.8695826 , 3.10034075, ..., 3.39997981, 1.75416737,
         2.11173819],
        [2.50827179, 2.47164675, 2.16751198, ..., 4.16734951, 1.66135962,
         1.64358464]]))

In [429]:
x = torch.FloatTensor(x_test[:,:-1])
c = torch.FloatTensor(c_hat_test)
# evaluate
new_model.eval()
with torch.no_grad():  # no grad
    cp = new_model(x) #.to("cpu").detach().numpy()

sol_cp, obj_cp = shortest_path_solver(cp, 5)
sol_c, obj_c = shortest_path_solver(c, 5)
out_obj_cp = torch.sum(sol_cp * c)
out_obj_c = torch.sum(sol_c * c)

torch.sum(out_obj_cp - out_obj_c).item()/(torch.sum(abs(out_obj_c)).item() + 1e-7)


0.08560629393454147

New Regret

In [404]:
old_cp = old_model(torch.tensor(final_data_test['X'], dtype=torch.float32))
new_cp = new_model(torch.tensor(final_data_test['X'], dtype=torch.float32))
(new_cp - old_cp).abs().mean()

False

In [436]:
new_reg_func_old_mod = decision_regret(pred_cost=old_cp,
                true_cost=final_data_test['true_cost'],
                true_obj=final_data_test['true_obj'],
                optmodel=optmodel,
                minimize=True)

new_reg_func_new_mod = decision_regret(pred_cost=new_cp,
                true_cost=final_data_test['true_cost'],
                true_obj=final_data_test['true_obj'],
                optmodel=optmodel,
                minimize=True)

In [437]:
new_reg_func_old_mod, new_reg_func_new_mod

(0.04638751375241514, 0.04638751375241514)

In [430]:
final_data_test.keys()

dict_keys(['X', 'true_cost', 'true_sol', 'true_obj'])

In [410]:
w_hat, z_hat = optmodel(new_cp)
    
# To ensure consistency, convert everything into a pytorch tensor
w_hat = torch.tensor(w_hat, dtype=torch.float32)
z_hat = torch.tensor(z_hat, dtype=torch.float32)
true_cost = torch.tensor(final_data_test['true_cost'], dtype=torch.float32)
true_obj = torch.tensor(final_data_test['true_obj'], dtype=torch.float32)

# objective value of pred_cost induced solution (w_hat) based on true cost
obj_hat = (w_hat * true_cost).sum(axis=1, keepdim=True)
    
regret = (obj_hat - true_obj).sum()
minimize=True
if not minimize:
    regret = -regret

opt_obj_sum = torch.sum(torch.abs(true_obj)).item() + 1e-7
regret.item() / opt_obj_sum

/tmp/SLURM_27423203/ipykernel_37376/2143074558.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w_hat = torch.tensor(w_hat, dtype=torch.float32)
/tmp/SLURM_27423203/ipykernel_37376/2143074558.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  z_hat = torch.tensor(z_hat, dtype=torch.float32)
/tmp/SLURM_27423203/ipykernel_37376/2143074558.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  true_obj = torch.tensor(final_data_test['true_obj'], dtype=torch.float32)


0.08560642662104108

Given same prediction - same optimal solution?

In [415]:
(sol_cp == w_hat).all()

tensor(True)

In [422]:
(obj_cp - z_hat).abs().max()

tensor(9.5367e-07)

In [424]:
true_obj, obj_c

(tensor([[14.0896],
         [14.6872],
         [16.2804],
         ...,
         [15.2295],
         [15.1068],
         [15.2983]]),
 tensor([[15.9502],
         [16.0000],
         [16.0000],
         ...,
         [16.0000],
         [16.0000],
         [16.0000]]))

# Initialize Data

# Training Loop

In [202]:
# Prediction Model
pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])
initial_weights = copy.deepcopy(pred_model.state_dict())

model = LinearRegression(input_dim=train_dict_old['X'].shape[1],
                 output_dim=train_dict_old['true_cost'].shape[1])
model.load_state_dict(copy.deepcopy(initial_weights))

model_2 = LinearRegression(input_dim=train_dict_old['X'].shape[1],
                 output_dim=train_dict_old['true_cost'].shape[1])
model_2.load_state_dict(copy.deepcopy(initial_weights))


<All keys matched successfully>

In [203]:
pred_model.linear.weight[0]

tensor([ 0.0941, -0.3390, -0.3272,  0.0868,  0.0219, -0.2847],
       grad_fn=<SelectBackward0>)

### New Data

In [175]:
generated_data = genData(num_data=num_data+200,
        num_features=num_feat, 
        grid=grid, 
        deg=deg, 
        noise_type=ep_type,
        noise_width=e,
        seed=indices_arr[trial],     
        plant_edges=plant_edge,
        planted_good_pwl_params=planted_good_pwl_params,
        planted_bad_pwl_params=planted_bad_pwl_params)
sol, obj = shortest_path_solver(costs=generated_data['cost'], size=5)
final_data = {'X':generated_data['feat'],
              'true_cost':generated_data['cost'],
              'true_sol':sol,
              'true_obj':obj}

# Splitting each input in the same way
train_dict = {}
val_dict = {}

for key, value in final_data.items():
    train_data, val_data = train_test_split(value, test_size=200, random_state=42)
    train_dict[key] = train_data
    val_dict[key] = val_data
    
# test data
generated_data_test = genData(num_data=10000,
        num_features=num_feat, 
        grid=grid, 
        deg=deg, 
        noise_type=ep_type,
        noise_width=e,
        seed=indices_arr_test[trial],     
        plant_edges=plant_edge,
        planted_good_pwl_params=planted_good_pwl_params,
        planted_bad_pwl_params=planted_bad_pwl_params)
sol_test, obj_test = shortest_path_solver(costs=generated_data_test['cost'], size=5)
final_data_test = {'X':generated_data_test['feat'],
              'true_cost':generated_data_test['cost'],
              'true_sol':sol_test,
              'true_obj':obj_test}

2024-11-13 13:15:41,508 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 10 11 12 13 14 17 18 19 20 21 22 25 26 27 28 29 32 33 34 37]
2024-11-13 13:15:41,508 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 10 11 12 13 14 17 18 19 20 21 22 25 26 27 28 29 32 33 34 37]
2024-11-13 13:15:41,513 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.0
2024-11-13 13:15:41,513 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.0
2024-11-13 13:15:41,513 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.55
2024-11-13 13:15:41,513 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.55
2024-11-13 13:15:41,538 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 

In [204]:
# Deep copy the model
print(model.linear.weight[0])
# optimization solver
optmodel = partial(shortest_path_solver,size=5)

# loss function
loss_fn = SPOPlus(optmodel=optmodel)

metrics, trained_model = train(pred_model=pred_model,
                optmodel=optmodel,
                loss_fn=loss_fn,
                train_data_dict=train_dict,
                val_data_dict=val_dict,
                dataloader_params={'batch_size':200, 'shuffle':True},
                num_epochs=100,
                lr=0.01,
                scheduler_params=None, #{'step_size': 10, 'gamma': 0.1},
                minimization=True)

tensor([ 0.0941, -0.3390, -0.3272,  0.0868,  0.0219, -0.2847],
       grad_fn=<SelectBackward0>)


100%|██████████| 1/1 [00:00<00:00, 676.17it/s]
2024-11-13 13:23:31,393 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 14.1853666305542, val_metric: 0.2569626234293161
2024-11-13 13:23:31,393 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 14.1853666305542, val_metric: 0.2569626234293161
100%|██████████| 1/1 [00:00<00:00, 688.38it/s]
2024-11-13 13:23:31,401 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 13.587915420532227, val_metric: 0.2531696263922039
2024-11-13 13:23:31,401 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 13.587915420532227, val_metric: 0.2531696263922039
100%|██████████| 1/1 [00:00<00:00, 705.87it/s]
2024-11-13 13:23:31,410 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 13.024239540100098, val_metric: 0.2137474692494163
2024-11-13 13:23:31,410 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 13.024239540100098, val_metric: 0.2137474692494163
100%|██████████| 

In [212]:
test_regret = calc_test_regret(pred_model=trained_model,
                               test_data_dict=final_data_test, #final_data_te,
                               optmodel=optmodel)
print(test_regret)

0.08402777103613965


### Old Data

In [214]:
feat, cost_true, cost, eps = sp_grid_mike_neurips.genData(num_data + 200, 
                                                     num_feat, 
                                                     grid, 
                                                     ep_type, 
                                                     deg, 
                                                     e, 
                                                     seed=indices_arr[trial])
x_train, x_val, c_train, c_val = train_test_split(feat, cost, test_size=200, random_state=42)
x_test, c_test, c_hat_test, eps_test = sp_grid_mike_neurips.genData(10000, 
                                                          num_feat, 
                                                          grid, 
                                                          ep_type, 
                                                          deg, 
                                                          e, 
                                                          seed=indices_arr_test[trial])

# train
sol_tr, obj_tr = shortest_path_solver(costs=c_train, size=5)
train_dict_old = {'X':x_train, #[:,:-1],
              'true_cost':c_train,
              'true_sol':sol_tr,
              'true_obj':obj_tr}
# val
sol_v, obj_v = shortest_path_solver(costs=c_val, size=5)
val_dict_old = {'X':x_val, #[:,:-1],
              'true_cost':c_val,
              'true_sol':sol_v,
              'true_obj':obj_v}

# test
sol_te, obj_te = shortest_path_solver(costs=c_hat_test, size=5)
final_data_te = {'X':x_test, #[:,:-1],
              'true_cost':c_hat_test,
              'true_sol':sol_te,
              'true_obj':obj_te}

In [215]:
# Deep copy the model
model_2 = LinearRegression(input_dim=train_dict_old['X'].shape[1],
                 output_dim=train_dict_old['true_cost'].shape[1])
print(model_2.linear.weight[0])
# optimization solver
optmodel_2 = partial(shortest_path_solver,size=5)

# loss function
loss_fn_2 = SPOPlus(optmodel=optmodel)

metrics_2, trained_model_2 = train(pred_model=model_2,
                optmodel=optmodel_2,
                loss_fn=loss_fn_2,
                train_data_dict=train_dict_old,
                val_data_dict=val_dict_old,
                dataloader_params={'batch_size':200, 'shuffle':True},
                num_epochs=100,
                lr=0.01,
                scheduler_params=None, #{'step_size': 10, 'gamma': 0.1},
                minimization=True)

tensor([-0.2872, -0.1867, -0.1686,  0.0504,  0.2973, -0.0260, -0.0786],
       grad_fn=<SelectBackward0>)


100%|██████████| 1/1 [00:00<00:00, 586.45it/s]
2024-11-13 13:26:29,649 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 15.820372581481934, val_metric: 0.5183820143070256
2024-11-13 13:26:29,649 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 15.820372581481934, val_metric: 0.5183820143070256
100%|██████████| 1/1 [00:00<00:00, 681.23it/s]
2024-11-13 13:26:29,662 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 14.878893852233887, val_metric: 0.5010580228972782
2024-11-13 13:26:29,662 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 14.878893852233887, val_metric: 0.5010580228972782
100%|██████████| 1/1 [00:00<00:00, 687.82it/s]
2024-11-13 13:26:29,670 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 13.96707534790039, val_metric: 0.4288215955036147
2024-11-13 13:26:29,670 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 13.96707534790039, val_metric: 0.4288215955036147
100%|██████████

In [216]:
test_regret = calc_test_regret(pred_model=trained_model_2,
                               test_data_dict=final_data_te, #final_data_te,
                               optmodel=optmodel)
print(test_regret)

0.07610093450950099


In [40]:
c_train.shape

torch.Size([200, 40])

In [22]:
feat.shape

torch.Size([400, 7])

In [439]:
generated_data = genData(num_data=num_data+200,
        num_features=num_feat, 
        grid=grid, 
        deg=deg, 
        noise_type=ep_type,
        noise_width=e,
        seed=indices_arr[trial],     
        plant_edges=plant_edge,
        planted_good_pwl_params=planted_good_pwl_params,
        planted_bad_pwl_params=planted_bad_pwl_params)
sol, obj = shortest_path_solver(costs=generated_data['cost'], size=5)
final_data = {'X':generated_data['feat'],
              'true_cost':generated_data['cost'],
              'true_sol':sol,
              'true_obj':obj}

# 1. Data - features, costs
# 2. Prediction model/plugin
# 3. solver/optimization
# 4. loss function (optional - try out everything)

# def aggregateData(X, y, loss_fn, solver):

#     if loss_fn...
#         sol, obj = solver(costs=y)
#         final_data = {'X':X,
#                       'true_cost':y,
#                       'true_sol':sol,
#                       'true_obj':obj}


# custom user - unique loss function
# TODO: how to combine it with the pipeline?
# 1. Single pipeline
# 2. data first, then losses

# experiment code loop - call the pipeline, custom behavior..
# single pipeline

# custom behavior - pipeline


# test data
generated_data_test = genData(num_data=10000,
        num_features=num_feat, 
        grid=grid, 
        deg=deg, 
        noise_type=ep_type,
        noise_width=e,
        seed=indices_arr[trial],     
        plant_edges=plant_edge,
        planted_good_pwl_params=planted_good_pwl_params,
        planted_bad_pwl_params=planted_bad_pwl_params)
sol_test, obj_test = shortest_path_solver(costs=generated_data_test['cost_true'], size=5)
final_data_test = {'X':generated_data_test['feat'],
              'true_cost':generated_data_test['cost_true'],
              'true_sol':sol_test,
              'true_obj':obj_test}

2024-11-13 16:25:28,526 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 10 11 12 13 14 17 18 19 20 21 22 25 26 27 28 29 32 33 34 37]
2024-11-13 16:25:28,526 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 10 11 12 13 14 17 18 19 20 21 22 25 26 27 28 29 32 33 34 37]
2024-11-13 16:25:28,527 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.0
2024-11-13 16:25:28,527 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.0
2024-11-13 16:25:28,528 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.55
2024-11-13 16:25:28,528 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.55
2024-11-13 16:25:28,554 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 

Split data into train and validation set

In [440]:
# Splitting each input in the same way
train_dict = {}
val_dict = {}

for key, value in final_data.items():
    train_data, val_data = train_test_split(value, test_size=200, random_state=42)
    train_dict[key] = train_data
    val_dict[key] = val_data


# double checking the splitting is done correctly
# Splitting indices
indices = np.arange(len(next(iter(final_data.values()))))
train_indices, val_indices = train_test_split(indices, test_size=200, random_state=42)

for key in final_data.keys():
    train_same = (final_data[key][train_indices] == train_dict[key]).all()
    val_same = (final_data[key][val_indices] == val_dict[key]).all()
    print(f'{key}: {train_same}, {val_same}')

X: True, True
true_cost: True, True
true_sol: True, True
true_obj: True, True


# Training Loop

Initialize Inputs to Trainining Loop

In [441]:
# Prediction Model
pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])

# optimization solver
optmodel = partial(shortest_path_solver,size=5)

# loss function
loss_fn = SPOPlus(optmodel=optmodel)

# training, validation data
train_data_dict = train_dict
val_data_dict = val_dict

In [442]:
metrics, trained_model = train(pred_model=pred_model,
                optmodel=optmodel,
                loss_fn=loss_fn,
                train_data_dict=train_data_dict,
                val_data_dict=val_data_dict,
                dataloader_params={'batch_size':200, 'shuffle':True},
                num_epochs=100,
                lr=0.01,
                scheduler_params=None, #{'step_size': 10, 'gamma': 0.1},
                minimization=True)

100%|██████████| 1/1 [00:00<00:00, 673.24it/s]
2024-11-13 16:25:40,572 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 14.039381980895996, val_metric: 0.364844584815951
2024-11-13 16:25:40,572 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 14.039381980895996, val_metric: 0.364844584815951
100%|██████████| 1/1 [00:00<00:00, 235.64it/s]
2024-11-13 16:25:40,582 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 13.375387191772461, val_metric: 0.35583657445013506
2024-11-13 16:25:40,582 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 13.375387191772461, val_metric: 0.35583657445013506
100%|██████████| 1/1 [00:00<00:00, 729.82it/s]
2024-11-13 16:25:40,590 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 12.743906021118164, val_metric: 0.3256780603369004
2024-11-13 16:25:40,590 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 12.743906021118164, val_metric: 0.3256780603369004
100%|████████

In [443]:
metrics

,epoch,train_loss,val_metric
0,0,14.039382,0.364845
1,1,13.375387,0.355837
2,2,12.743906,0.325678
3,3,12.143230,0.313259
4,4,11.577141,0.312537
...,...,...,...
95,95,4.125385,0.206730
96,96,4.120779,0.196237
97,97,4.116006,0.191460
98,98,4.111292,0.209852


In [138]:
val_dict['X'].shape

(200, 6)

In [444]:
val_regret = calc_test_regret(pred_model=trained_model,
                               test_data_dict=val_dict,
                               optmodel=optmodel)
print(val_regret)

0.08272489650178559


/home1/yongpeng/decision-focused-learning/src/decision_learning/modeling/val_metrics.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w_hat = torch.tensor(w_hat, dtype=torch.float32)
/home1/yongpeng/decision-focused-learning/src/decision_learning/modeling/val_metrics.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  z_hat = torch.tensor(z_hat, dtype=torch.float32)
/home1/yongpeng/decision-focused-learning/src/decision_learning/modeling/val_metrics.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  true_obj = torch.tensor(true_obj

In [445]:
test_regret = calc_test_regret(pred_model=trained_model,
                               test_data_dict=final_data_test, #final_data_te,
                               optmodel=optmodel)
print(test_regret)

0.045393637861943975


## MSE

In [27]:
# Prediction Model
pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])

# optimization solver
optmodel = partial(shortest_path_solver,size=5)

# loss function
loss_fn = nn.MSELoss()

# training, validation data
train_data_dict = train_dict
train_data_dict.update({'target':train_dict['true_cost']}) # extra input key needed for loss function

val_data_dict = val_dict

In [29]:
metrics, trained_model = train(pred_model=pred_model,
                optmodel=optmodel,
                loss_fn=loss_fn,
                train_data_dict=train_data_dict,
                val_data_dict=val_data_dict,
                num_epochs=100,
                lr=0.1,
                scheduler_params={'step_size': 10, 'gamma': 0.1},
                minimization=True)

100%|██████████| 12/12 [00:00<00:00, 3463.98it/s]
2024-11-06 09:09:21,646 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 1.8876804921362136, val_metric: 0.2392113968593179
2024-11-06 09:09:21,646 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 1.8876804921362136, val_metric: 0.2392113968593179
100%|██████████| 12/12 [00:00<00:00, 3477.38it/s]
2024-11-06 09:09:21,683 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 0.765940973493788, val_metric: 0.2152039387182229
2024-11-06 09:09:21,683 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 0.765940973493788, val_metric: 0.2152039387182229
100%|██████████| 12/12 [00:00<00:00, 3445.72it/s]
2024-11-06 09:09:21,720 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 0.6751075949933794, val_metric: 0.21910987352311345
2024-11-06 09:09:21,720 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 0.6751075949933794, val_metric: 0.21910987352311345
100%

In [30]:
test_regret = calc_test_regret(pred_model=pred_model,
                               test_data_dict=final_data_test,
                               optmodel=optmodel)
print(test_regret)

0.08562925913696434


## Cosine Loss

In [40]:
# Prediction Model
pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])

# optimization solver
optmodel = partial(shortest_path_solver,size=5)

# loss function
loss_fn = nn.CosineEmbeddingLoss()

# training, validation data
train_data_dict = train_dict
train_data_dict.update({'input2':train_dict['true_cost'], 
                       'target':torch.ones(train_dict['true_cost'].shape[0])}) # extra input key needed for loss function

val_data_dict = val_dict

In [41]:
metrics, trained_model = train(pred_model=pred_model,
                optmodel=optmodel,
                loss_fn=loss_fn,
                train_data_dict=train_data_dict,
                val_data_dict=val_data_dict,
                num_epochs=100,
                lr=0.1,
                scheduler_params={'step_size': 10, 'gamma': 0.1},
                minimization=True)

100%|██████████| 12/12 [00:00<00:00, 3458.51it/s]
2024-11-06 09:18:08,838 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 0.07857334117094676, val_metric: 0.15394632031310854
2024-11-06 09:18:08,838 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 0.07857334117094676, val_metric: 0.15394632031310854
100%|██████████| 12/12 [00:00<00:00, 3487.02it/s]
2024-11-06 09:18:08,884 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 0.023548006556100315, val_metric: 0.16055482282145492
2024-11-06 09:18:08,884 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 0.023548006556100315, val_metric: 0.16055482282145492
100%|██████████| 12/12 [00:00<00:00, 3498.65it/s]
2024-11-06 09:18:08,931 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 0.022603477413455645, val_metric: 0.1482329970276144
2024-11-06 09:18:08,931 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 0.022603477413455645, val_metric: 0.1482329

In [42]:
test_regret = calc_test_regret(pred_model=pred_model,
                               test_data_dict=final_data_test,
                               optmodel=optmodel)
print(test_regret)

0.06871183614832832


## PG Loss

In [56]:
import decision_learning.modeling.loss
importlib.reload(decision_learning.modeling.loss)
from decision_learning.modeling.loss import PG_Loss

In [50]:
# optimization solver
optmodel = partial(shortest_path_solver,size=5)

# training, validation data
train_data_dict = train_dict
val_data_dict = val_dict

PG Loss Parameters

In [54]:
h_arr = [num_data**-.125, num_data**-.25, num_data**-.5, num_data**-1]
fin_diff_sch = ['B', 'F', 'C']

In [57]:
# loop over PG loss params
PG_results = []
for h in h_arr:
    for f in fin_diff_sch:
        
        print(h, f)
        
        # --- Need to instantiate new loss function and prediction model for each possible setting of PG loss ---   
        # loss function
        loss_fn = PG_Loss(optmodel=optmodel,
                         h=h,
                         finite_diff_type=f)
        # Prediction Model
        pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                         output_dim=train_dict['true_cost'].shape[1])
        # ----------------------------------------------------------------------------------------------------------------
        
        metrics, trained_model = train(pred_model=pred_model,
                        optmodel=optmodel,
                        loss_fn=loss_fn,
                        train_data_dict=train_data_dict,
                        val_data_dict=val_data_dict,
                        num_epochs=100,
                        lr=0.1,
                        scheduler_params={'step_size': 10, 'gamma': 0.1},
                        minimization=True)
        test_regret = calc_test_regret(pred_model=pred_model,
                               test_data_dict=final_data_test,
                               optmodel=optmodel)

        # add current setting results to overall results
        cur_res = {'h':h, 'finite_diff_sch': f, 'test_regret': test_regret}
        PG_results.append(cur_res)

# convert to pandas dataframe
PG_results = pd.DataFrame(PG_results)


0.3976353643835253 B


100%|██████████| 12/12 [00:00<00:00, 3474.98it/s]
2024-11-06 10:50:49,084 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 23.011095428466795, val_metric: 0.6255135366346355
2024-11-06 10:50:49,084 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 23.011095428466795, val_metric: 0.6255135366346355
100%|██████████| 12/12 [00:00<00:00, 3451.16it/s]
2024-11-06 10:50:49,215 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 23.265939500596787, val_metric: 0.6662045242176788
2024-11-06 10:50:49,215 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 23.265939500596787, val_metric: 0.6662045242176788
100%|██████████| 12/12 [00:00<00:00, 3441.72it/s]
2024-11-06 10:50:49,345 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 23.35850109524197, val_metric: 0.6505704453340708
2024-11-06 10:50:49,345 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 23.35850109524197, val_metric: 0.6505704453340708
100%|█

0.3976353643835253 F


100%|██████████| 12/12 [00:00<00:00, 3494.53it/s]
2024-11-06 10:51:02,077 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 21.870707575480143, val_metric: 0.663972930745743
2024-11-06 10:51:02,077 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 21.870707575480143, val_metric: 0.663972930745743
100%|██████████| 12/12 [00:00<00:00, 3478.83it/s]
2024-11-06 10:51:02,207 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 22.582617653740776, val_metric: 0.6422090023064015
2024-11-06 10:51:02,207 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 22.582617653740776, val_metric: 0.6422090023064015
100%|██████████| 12/12 [00:00<00:00, 3482.20it/s]
2024-11-06 10:51:02,336 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 22.654586283365884, val_metric: 0.6620179739716615
2024-11-06 10:51:02,336 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 22.654586283365884, val_metric: 0.6620179739716615
100%|█

0.3976353643835253 C


...

0.000625 F


100%|██████████| 12/12 [00:00<00:00, 3539.25it/s]
2024-11-06 10:52:59,903 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 21.449234347873265, val_metric: 0.3697886319169076
2024-11-06 10:52:59,903 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 21.449234347873265, val_metric: 0.3697886319169076
100%|██████████| 12/12 [00:00<00:00, 3539.00it/s]
2024-11-06 10:53:00,036 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 18.69632191128201, val_metric: 0.3699657493483205
2024-11-06 10:53:00,036 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 18.69632191128201, val_metric: 0.3699657493483205
100%|██████████| 12/12 [00:00<00:00, 3537.26it/s]
2024-11-06 10:53:00,168 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 18.69419576856825, val_metric: 0.3757900466261107
2024-11-06 10:53:00,168 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 18.69419576856825, val_metric: 0.3757900466261107
100%|███

0.000625 C


100%|██████████| 12/12 [00:00<00:00, 3523.39it/s]
2024-11-06 10:53:12,740 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 21.136268191867405, val_metric: 0.5513897522055999
2024-11-06 10:53:12,740 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 21.136268191867405, val_metric: 0.5513897522055999
100%|██████████| 12/12 [00:00<00:00, 3541.49it/s]
2024-11-06 10:53:12,869 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 20.29042994181315, val_metric: 0.5520808371039588
2024-11-06 10:53:12,869 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 20.29042994181315, val_metric: 0.5520808371039588
100%|██████████| 12/12 [00:00<00:00, 3535.77it/s]
2024-11-06 10:53:12,998 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 20.282528347439236, val_metric: 0.5514522970361518
2024-11-06 10:53:12,998 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 20.282528347439236, val_metric: 0.5514522970361518
100%|█

NameError: name 'pd' is not defined

In [61]:
PG_results

,h,finite_diff_sch,test_regret
0,0.397635,B,0.766241
1,0.397635,F,0.703595
2,0.397635,C,0.760432
3,0.158114,B,0.742369
4,0.158114,F,0.732630
5,0.158114,C,0.732590
6,0.025000,B,0.715896
7,0.025000,F,0.711148
8,0.025000,C,0.703984
9,0.000625,B,0.572492
